<h2>--- Day 9: Smoke Basin ---</h2>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/oddrationale/AdventOfCode2021FSharp/main?urlpath=lab%2Ftree%2FDay09.ipynb)

<p>These caves seem to be <a href="https://en.wikipedia.org/wiki/Lava_tube" target="_blank">lava tubes</a>. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly <span title="This was originally going to be a puzzle about watersheds, but we're already under water.">settles like rain</span>.</p>
<p>If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).</p>
<p>Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:</p>
<pre><code>2<em>1</em>9994321<em>0</em>
3987894921
98<em>5</em>6789892
8767896789
989996<em>5</em>678
</code></pre>
<p>Each number corresponds to the height of a particular location, where <code>9</code> is the highest and <code>0</code> is the lowest a location can be.</p>
<p>Your first goal is to find the <em>low points</em> - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)</p>
<p>In the above example, there are <em>four</em> low points, all highlighted: two are in the first row (a <code>1</code> and a <code>0</code>), one is in the third row (a <code>5</code>), and one is in the bottom row (also a <code>5</code>). All other locations on the heightmap have some lower adjacent location, and so are not low points.</p>
<p>The <em>risk level</em> of a low point is <em>1 plus its height</em>. In the above example, the risk levels of the low points are <code>2</code>, <code>1</code>, <code>6</code>, and <code>6</code>. The sum of the risk levels of all low points in the heightmap is therefore <code><em>15</em></code>.</p>
<p>Find all of the low points on your heightmap. <em>What is the sum of the risk levels of all low points on your heightmap?</em></p>

In [ ]:
let input = 
    File.ReadAllLines @"input/09.test.txt"
    |> Array.map (fun line -> 
        line
        |> Seq.map (fun c -> c |> string |> int)
        |> Array.ofSeq)

In [ ]:
let lowPoints (input: int array array) = 
    let maxY = input |> Seq.length
    let maxX = input |> Seq.head |> Seq.length
    let directions = [
        ( 1,  0) // right
        ( 0,  1) // down
        (-1,  0) // left
        ( 0, -1) // up
    ]
    
    input
    |> Seq.mapi (fun y line -> 
        line
        |> Seq.mapi (fun x h ->
            x, y, h))
    |> Seq.collect id
    |> Seq.filter (fun (x, y, h) -> 
        directions
        |> Seq.map (fun (x', y') -> (x + x', y + y'))
        |> Seq.filter (fun (x', y') -> -1 < x' && x' < maxX && -1 < y' && y' < maxY)
        |> Seq.map (fun (x', y') -> input.[y'].[x'])
        |> fun neighbors -> h < (neighbors |> Seq.min))
    

In [ ]:
#!time
input
|> lowPoints
|> Seq.map (fun (_, _, h) -> h + 1)
|> Seq.sum

15

Wall time: 17.524ms

<h2 id="part2">--- Part Two ---</h2>

<p>Next, you need to find the largest basins so you know what areas are most important to avoid.</p>
<p>A <em>basin</em> is all locations that eventually flow downward to a single low point. Therefore, every low point has a basin, although some basins are very small. Locations of height <code>9</code> do not count as being in any basin, and all other locations will always be part of exactly one basin.</p>
<p>The <em>size</em> of a basin is the number of locations within the basin, including the low point. The example above has four basins.</p>
<p>The top-left basin, size <code>3</code>:</p>
<pre><code><em>21</em>99943210
<em>3</em>987894921
9856789892
8767896789
9899965678
</code></pre>
<p>The top-right basin, size <code>9</code>:</p>
<pre><code>21999<em>43210</em>
398789<em>4</em>9<em>21</em>
985678989<em>2</em>
8767896789
9899965678
</code></pre>
<p>The middle basin, size <code>14</code>:</p>
<pre><code>2199943210
39<em>878</em>94921
9<em>85678</em>9892
<em>87678</em>96789
9<em>8</em>99965678
</code></pre>
<p>The bottom-right basin, size <code>9</code>:</p>
<pre><code>2199943210
3987894921
9856789<em>8</em>92
876789<em>678</em>9
98999<em>65678</em>
</code></pre>
<p>Find the three largest basins and multiply their sizes together. In the above example, this is <code>9 * 14 * 9 = <em>1134</em></code>.</p>
<p><em>What do you get if you multiply together the sizes of the three largest basins?</em></p>

In [ ]:
let basin (input: int array array) (x, y, h) =
    let maxY = input |> Seq.length
    let maxX = input |> Seq.head |> Seq.length
    let directions = [
        ( 1,  0) // right
        ( 0,  1) // down
        (-1,  0) // left
        ( 0, -1) // up
    ]
    
    let rec loop discovered (x', y', h') = 
        let neighbors = 
            directions
            |> Seq.map (fun (x'', y'') -> x' + x'', y' + y'')
            |> Seq.filter (fun (x'', y'') -> -1 < x'' && x'' < maxX && -1 < y'' && y'' < maxY)
            |> Seq.map (fun (x'', y'') -> x'', y'', input.[y''].[x''])
            |> Seq.filter (fun (_, _, h'') -> h'' <> 9)
            |> Seq.except discovered
            |> Set.ofSeq
        if neighbors |> Set.isEmpty then
            discovered
        else
            neighbors
            |> Seq.map (fun (x'', y'', h'') -> loop (discovered + neighbors) (x'', y'', h''))
            |> Seq.collect id
            |> Set.ofSeq 
            
    loop ([(x, y, h)] |> Set.ofSeq) (x, y, h)

In [ ]:
#!time
input
|> lowPoints
|> Seq.map (fun (x, y, h) -> basin input (x, y, h))
|> Seq.map (fun basins -> basins |> Seq.length)
|> Seq.sortDescending
|> Seq.truncate 3
|> Seq.reduce (*)

1134

Wall time: 37.1418ms

[Prev](Day08.ipynb) | [Next](Day10.ipynb)